In [1]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv("C:\\Users\\Sushant-Chaudhary\\Downloads\\songdata.csv")
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [14]:
df.shape

(57650, 4)

In [15]:
df=df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [17]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [18]:
df['text'][0]

"will their children cry   when their mother dies   and in the autumn of their lives   will they feel the same?      [reversed lyrics 2:03-2:22]      the longing for me   comin' down like hail   why did you leave me?      come here inside   and tell me how   how in my final days   you found around      your concern   the signal for us to end our lives   to wipe beneath the watches   and you wait forever      the night comes high   your weakness   storing trace on your eyes   so i left you alone   we all left you alone      [reversed lyrics 7:21-7:24]      lock the children away from harm   they'll lock all your reason why   seeking (?) with a dagger   skin is blocked by the years of trial      you felt abandoned in the fog of flesh   sitting in place from the dead   awaiting the face of the moon to ascend   you follow the siren in your head      in your head  "

In [37]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens=nltk.word_tokenize(txt)
    stemming=[stemmer.stem(word) for word in tokens]
    return " ".join(stemming)

In [38]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Sushant-
[nltk_data]     Chaudhary\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [44]:
similarity[0]

array([1.        , 0.05887988, 0.02346008, ..., 0.05989863, 0.06384585,
       0.02459187])

In [90]:
df['song'][1]


'Throw The Chains Away'

In [88]:
df[df['song']=='']

,artist,song,text


In [86]:
def recommendation(song_df):
    filtered_rows = df[df['song'] == song_df]

    if not filtered_rows.empty:
        idx = filtered_rows.index[0]
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

        songs = []
        for m_id in distances[1:21]:
            songs.append(df.iloc[m_id[0]].song)

        return songs
    else:
        print(f"No song with the name '{song_df}' found in the DataFrame.")
        return []


In [91]:
recommendation('Throw The Chains Away')

['Man In Chains',
 'Draw The Line',
 'Ball And Chain',
 'Ring Them Bells',
 "Who'll Be The Next In Line",
 'Let It Ring',
 'Broke Away',
 "Takin' Back My Love",
 "I'll Be There For You",
 'Late',
 'My Resurrection',
 "I'll Be There",
 "It Ain't Over Yet",
 'Too Late',
 'Rip',
 'Break The Ice',
 "I'll Be Around",
 'Prisoner Of Love',
 'Sold My Soul To Rock And Roll',
 'Steel Bars']

In [59]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))